# Assignment: Conversation Management & Classification using Groq API

# This Colab notebook implements the two tasks:
#   1. Conversation Management & Summarization
#   2. JSON Schema Classification & Extraction
# Using Groq API (OpenAI-compatible SDK).




In [1]:

!pip install openai

import os
import json
from openai import OpenAI

In [ ]:






#  Set your API Key

os.environ["OPENAI_API_KEY"] = "YOURGROQAPIKEYHERE"# insert your GROQ API KEY
client = OpenAI(api_key=os.environ["OPENAI_API_KEY"], base_url="https://api.groq.com/openai/v1")


# Task 1: Conversation Management with Summarization


conversation_history = []
summary_counter = 0

def add_message(role, content):
    """Add a message to the conversation history."""
    conversation_history.append({"role": role, "content": content})

def truncate_history_by_turns(n=5):
    """Keep only the last n turns (messages)."""
    global conversation_history
    conversation_history = conversation_history[-n:]

def truncate_history_by_chars(max_chars=200):
    """Keep conversation within a maximum character length."""
    global conversation_history
    text = ""
    truncated = []
    for msg in reversed(conversation_history):
        if len(text) + len(msg["content"]) <= max_chars:
            truncated.insert(0, msg)
            text += msg["content"]
        else:
            break
    conversation_history[:] = truncated

def summarize_history():
    """Call Groq API to summarize conversation so far."""
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[
            {"role": "system", "content": "Summarize the following conversation in 2-3 sentences."},
            {"role": "user", "content": str(conversation_history)}
        ]
    )
    summary = response.choices[0].message.content

    summarized = [{"role": "system", "content": f"Summary: {summary}"}]
    if conversation_history:
        summarized.append(conversation_history[-1])
    return summarized

def run_conversation(user_input, k=3):
    """Simulate conversation with periodic summarization every k turns."""
    global summary_counter, conversation_history
    add_message("user", user_input)


    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=conversation_history
    )
    assistant_reply = response.choices[0].message.content
    add_message("assistant", assistant_reply)

    summary_counter += 1

    # Every k turns, summarize
    if summary_counter % k == 0:
        conversation_history = summarize_history()

    return assistant_reply





print("=== Task 1 Demo: Interactive Conversation ===")

for i in range(3):  # simulate 3 turns
    user_msg = input("User: ")
    reply = run_conversation(user_msg, k=3)
    print("Assistant:", reply)
    print("Current Conversation History:", conversation_history)


print("\n--- Truncation Examples ---")

# Truncate by turns
truncate_history_by_turns(3)
print("After keeping last 3 messages:\n", conversation_history)

# Add another message
add_message("user", "By the way, I also need a cab from the airport.")

# Truncate by characters
truncate_history_by_chars(120)
print("After truncating to 120 characters:\n", conversation_history)



=== Task 1 Demo: Interactive Conversation ===
User: i WANT TO BUY A 350CC BIKE
Assistant: Buying a 350cc bike can be an exciting experience. Before we proceed, let's break down some key factors to consider:

**Reasons to buy a 350cc bike:**

1. **Power and performance**: 350cc bikes offer a good balance between power and handling, making them suitable for both city riding and highway cruising.
2. **Comfort**: 350cc bikes tend to have larger engines, which can provide a more comfortable riding experience, especially on long trips.
3. **Price**: 350cc bikes are generally more affordable than larger displacement bikes (500cc+), but may be pricier than smaller bikes (125cc-250cc).

**Types of 350cc bikes:**

1. **Adventure bikes**: These bikes, such as Triumph Tiger or BMW F 750 GS, are designed for off-road and on-road adventures.
2. **Cruisers**: Bikes like Harley-Davidson or Kawasaki Vulcan are perfect for laid-back, relaxed riding.
3. **Sport bikes**: These bikes, like Suzuki Hayabusa 

In [3]:

# Task 2: JSON Schema Classification & Information Extraction

schema = {
    "name": "extract_user_info",
    "description": "Extract user details from chat",
    "parameters": {
        "type": "object",
        "properties": {
            "name": {"type": "string", "description": "User's full name"},
            "email": {"type": "string", "description": "User's email address"},
            "phone": {"type": "string", "description": "User's phone number"},
            "location": {"type": "string", "description": "User's location"},
            "age": {"type": "string", "description": "User's age"}
        },
        "required": ["name", "email", "phone", "location", "age"]
    }
}

def extract_info_from_chat(chat_text):
    """Use Groq function calling to extract structured user info."""
    response = client.chat.completions.create(
        model="llama-3.1-8b-instant",
        messages=[{"role": "user", "content": chat_text}],
        functions=[schema],
        function_call={"name": "extract_user_info"}
    )
    result = response.choices[0].message

    if result.function_call:
        args = json.loads(result.function_call.arguments)


        if "age" in args:
            try:
                args["age"] = int(args["age"])
            except:
                args["age"] = None

        return args


    return None






# Task 2 (Interactive)

print("\n=== Task 2 Demo: Interactive JSON Schema Extraction ===")

while True:
    chat = input("Enter a chat (or type 'exit' to quit): ")
    if chat.lower() == "exit":
        break

    extracted = extract_info_from_chat(chat)
    print("Extracted JSON:", extracted)




=== Task 2 Demo: Interactive JSON Schema Extraction ===
Enter a chat (or type 'exit' to quit): Good morning, my name is John Doe, 28 years old, living in Mumbai. Email: john.doe@mail.com, phone 9000011111.
Extracted JSON: {'age': 28, 'email': 'john.doe@mail.com', 'location': 'Mumbai', 'name': 'John Doe', 'phone': '9000011111'}
Enter a chat (or type 'exit' to quit): exit
